In [ ]:
# !pip3 install chardet -i https://mirrors.aliyun.com/pypi/simple/

In [ ]:
import glob
import os
from tqdm import tqdm
import numpy as np
import shutil
import random

import IPython
def play_audio(data, rate):
    IPython.display.display(IPython.display.Audio(data=data,rate=rate))

In [ ]:
sr = 16000
result_base_dir = "./audio/20240815_spaccent/"
speakers = ["BWC", "LXC", "NCC", "TXHC"]

experiments = {
    "mushra_naturalness": [
        "source",
        "resynth",
        "pseudo",
        "ours",
        "ours-scale",
        "ours-control-scale",
        "baseline",
    ],
    "mushra_accentedness": [
        "source",
        "resynth",
        "pseudo",
        "ours",
        "ours-scale",
        "ours-control-scale",
        "baseline",
    ],
    "bws_similarity": [
        "source", # reference
        "ours", # Z
        "ours-scale", # Y
        "ours-control-scale", # X
        "baseline", # W
    ],
    "bws_naturalness": [ # duration control
        "ours-scale", # Z
        "ours-control-scale", # Y
    ],
}


import chardet
def openfile(txtfile):
    with open(txtfile, "rb") as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']

    with open(txtfile, "r", encoding=encoding) as f:
        content = f.readlines()
    return content
files = glob.glob("../../seq2seq-vc/datasetgeneration/LLM_responses/08-A_multi-lingual_text/*.npy")
files.sort()
groundtruth_dir = {}
for file in files:
    key = os.path.basename(file)[:-4]
    text = np.load(file, allow_pickle=True).item()["Original English"]
    groundtruth_dir[key] = text

In [ ]:
nsamples_per_test = {
    "mushra_naturalness": 15, # 15
    "mushra_accentedness": 15, # 15
    "bws_similarity": 15, # 15
    "bws_naturalness": 15, # 15
    # "mushra_naturalness": 3, # 15
    # "mushra_accentedness": 3, # 15
    # "bws_similarity": 5, # 15
    # "bws_naturalness": 5, # 15
}
testnames = {
    "mushra_naturalness": "Speech Naturalness Evaluation (MUSHRA)",
    "mushra_accentedness": "Speech Accentedness Evaluation (MUSHRA)",
    "bws_similarity": "Speaker Similarity Evaluation (BWS)", 
    "bws_naturalness": "Speech Naturalness Evaluation (BWS)", 
}
testids = {
    "mushra_naturalness": "MUSHRANAT",
    "mushra_accentedness": "MUSHRAACT",
    "bws_similarity": "BWSSIM", 
    "bws_naturalness": "BWSNAT", 
}

# MUSHRA

In [ ]:
save = True
for exid in ["mushra_naturalness", "mushra_accentedness"]:
# for exid in ["mushra_accentedness"]:
# for exid in ["mushra_naturalness"]:
    testname = testnames[exid]
    testmode = exid.split("_")[-1]
    testid = testids[exid]
    pngfile = "img/scale_natural.png" if testmode=="naturalness" else "img/scale_accent.png"
    # pngfile = "img/scale_abs.png" if testmode=="naturalness" else "img/scale_accent.png"
    def init():
        whole_text = f"""var TestConfig = {{
      "TestName": "{testname}",
      "RateScalePng": "{pngfile}",
      "RateScaleBgPng": "img/scale_abs_background.png",
      "RateMinValue": 0,
      "RateMaxValue": 100,
      "RateDefaultValue":0,
      "ShowFileIDs": false,
      "ShowResults": false,
      "LoopByDefault": false,
      "EnableABLoop": true,
      "EnableOnlineSubmission": false,
      "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
      "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
      "RandomizeTestOrder": true,
      "MaxTestsPerRun": {nsamples_per_test[exid]},
      "RequireMaxRating": false,
      "AudioRoot": "",
      "Testsets":
    ["""
        return whole_text

    whole_text = init()
    dirnames = experiments[exid]
    dirname = dirnames[0]
    for spk in speakers:
        filenames = [os.path.basename(a)[:-4] for a in glob.glob(result_base_dir+f"{dirname}_{spk}/*.wav")]
        filenames.sort()
        for b, basename in enumerate(filenames):
            _, name = basename.split("_")
            text = groundtruth_dir[basename]
            starter = f"""
      {{
        "Name": "{basename}",
        "TestID": "{testid}---{basename}---{spk}",
        "Text": "{text}",
        "Files": {{\n"""
            modelpaths = """"""
            for dirname in dirnames:
                modelpath = result_base_dir+f"{dirname}_{spk}/{basename}.wav"
                added = f'      "{dirname}": "{modelpath}",\n'
                modelpaths += added
            ##### One more sample #####
            np.random.seed(b)
            dirname = dirnames[np.random.randint(len(dirnames))]
            modelpath = result_base_dir+f"{dirname}_{spk}/{basename}.wav"
            added = f'      "{dirname}---2": "{modelpath}",\n'
            modelpaths += added
            ##########################
            modelpaths += f'    }}\n'
            modelpaths += f'  }},'
            whole_text += starter+modelpaths
    whole_text += "\n]\n}}"
    print(whole_text[:-1])
    savefile = f"./config/qibing_{exid}.js"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(whole_text[:-1])
        f.close()

# BWS Test

In [ ]:
instructions = {
    "bws_similarity":{
        "Low": "the 'LEAST' similar'",
        "High": "the 'MOST' similar'",
    },
    "bws_naturalness":{
        "Low": "the 'LEAST' natural'",
        "High": "the 'MOST' natural'",
    },
}

In [ ]:
save = True
bws_labels = ["V", "W", "X", "Y", "Z"][::-1]
evaluation_filenames = [os.path.basename(a)[:-4] for a in glob.glob("./audio/20240815_spaccent/source_TXHC/*")]
evaluation_filenames.sort()
for exid in ["bws_similarity", "bws_naturalness"]:
# for exid in ["bws_similarity"]:
    with_reference = exid in ["bws_similarity"]
    testname = testnames[exid]
    testmode = exid.split("_")[-1]
    testid = testids[exid]
    instruction = instructions[exid]
    def init():
        whole_text = f"""var TestConfig = {{
  "TestName": "{testname}",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": false,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "MaxTestsPerRun": {nsamples_per_test[exid]},
  "RandomizeTestOrder": true,
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
["""
        return whole_text

    whole_text = init()
    test_arrays = []
    modelnames = experiments[exid]
    lists = evaluation_filenames
    for basename in lists:
        for spk in speakers:
            test_arrays += [[basename, spk]]
    random.shuffle(test_arrays)
    for test_array in test_arrays:
        basename, spk = test_array
        text = groundtruth_dir[basename]
        if testmode=="accentedness":
            text = f"{text} --- Accent: {exid[0].upper()+exid[1:]}"
        starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "{testid}---{basename}---{spk}",
    "Text": "{text}",
    "LowText": "{instruction['Low']}",
    "HighText": "{instruction['High']}",
    "Length": {len(modelnames)-int(with_reference)},
    "Files": {{\n"""
        modelpaths = """"""
        for d, dirname in enumerate(modelnames):
            modelpath = result_base_dir+f"{dirname}_{spk}/{basename}.wav"
            if with_reference and d==0:
                label = "Reference"
            else:
                label = bws_labels[d-int(with_reference)]
            added = f'      "{label}": "{modelpath}",\n'
            modelpaths += added
        modelpaths += f'    }}\n'
        modelpaths += f'  }},'
        whole_text += starter+modelpaths
    whole_text += "\n]\n}}"
    print(whole_text[:-1])
    savefile = f"./config/qibing_{exid}.js"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(whole_text[:-1])
        f.close()

----
----
----
----
----
----
----
----
----
----
----

In [ ]:
num = 100000
start = 0
save = True
with_reference = False
evaluation_filenames = ["arctic_a0024", "arctic_a0029"]
bws_labels = ["W", "X", "Y", "Z"][::-1]
# for exid in ["bws_similarity", "bws_naturalness"]:
for exid in ["bws_similarity"]:
    testname = testnames[exid]
    testmode = exid.split("_")[-1]
    testid = testids[exid]
    def init():
        whole_text = f"""var TestConfig = {{
  "TestName": "{testname}",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": false,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
["""
        return whole_text

    whole_text = init()
    test_arrays = []
    modelnames = experiments[exid]
    for basename in evaluation_filenames:
        for spk in speakers:
            test_arrays += [[basename, spk]]
    random.shuffle(test_arrays)
    for test_array in test_arrays:
        basename, spk = test_array
        text = groundtruth_dir[basename]
        if testmode=="accentedness":
            text = f"{text} --- Accent: {exid[0].upper()+exid[1:]}"
        starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "{testid}---{basename}---{spk}",
    "Text": "{text}",
    "Length": {len(modelnames)-int(with_reference)},\n"""
        if with_reference:
            dirname = modelnames[0]
            modelpath = result_base_dir+f"{dirname}___{spk}/{basename}.wav"
            starter += f"""
    "Reference": "{modelpath}",
    "Files": {{\n"""[1:]
        else:
            starter += f"""
    "Files": {{\n"""[1:]
        modelpaths = """"""
        for d, dirname in enumerate(modelnames[int(with_reference):]):
            modelpath = result_base_dir+f"{dirname}___{spk}/{basename}.wav"
            added = f'      "{bws_labels[d]}": "{modelpath}",\n'
            modelpaths += added
        modelpaths += f'    }}\n'
        modelpaths += f'  }},'
        whole_text += starter+modelpaths
    whole_text += "\n]\n}}"
    print(whole_text[:-1])
    savefile = f"./config/qibing_{exid}.js"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(whole_text[:-1])
        f.close()

In [ ]:
num = 100000
start = 0
save = True
evaluation_filenames = ["arctic_a0024", "arctic_a0029"]
bws_labels = ["W", "X", "Y", "Z"][::-1]
# for exid in ["bws_similarity", "bws_naturalness"]:
for exid in ["bws_similarity"]:
    testname = testnames[exid]
    testmode = exid.split("_")[-1]
    testid = testids[exid]
    def init():
        whole_text = f"""var TestConfig = {{
  "TestName": "{testname}",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": false,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
["""
        return whole_text

    whole_text = init()
    test_arrays = []
    modelnames = experiments[exid]
    for basename in evaluation_filenames:
        for spk in speakers:
            test_arrays += [[basename, spk]]
    random.shuffle(test_arrays)
    for test_array in test_arrays:
        basename, spk = test_array
        text = groundtruth_dir[basename]
        if testmode=="accentedness":
            text = f"{text} --- Accent: {exid[0].upper()+exid[1:]}"
        starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "{testid}---{basename}---{spk}",
    "Text": "{text}",
    "Length": {len(modelnames)},
    "Files": {{\n"""
        modelpaths = """"""
        for d, dirname in enumerate(modelnames):
            dirname = f'{"___".join(dirname.split("/"))}'
            modelpath = result_base_dir+f"{dirname}___{spk}/{basename}.wav"
            added = f'      "{bws_labels[d]}": "{modelpath}",\n'
            modelpaths += added
        modelpaths += f'    }}\n'
        modelpaths += f'  }},'
        whole_text += starter+modelpaths
    whole_text += "\n]\n}}"
    print(whole_text[:-1])
    savefile = f"./config/qibing_{exid}.js"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(whole_text[:-1])
        f.close()

In [ ]:
import random

emos = ["Angry", "Happy", "Sad", "Surprise"]

def init():
    whole_text = """var TestConfig = {
  "TestName": "Utterance-level Emotion Intensity Evaluation",
  "LoopByDefault": false,
  "ShowFileIDs": false,
  "ShowResults": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "AudioRoot": "",
  "Testsets":
[\n"""
    return whole_text

evaluation_filenames = [
    # "0013_000731",
    "0014_000732",
    "0015_000726",
    # "0018_000037",
]

In [ ]:
num = 10000000
start = 0
whole_text = init()
result_dir = result_base_dir + f"utterance_remain/"
intensities = ["0.0", "0.6", "1.0"]
intensity_label = ["L", "M", "H"]
modelnames = list(set(experiments["main"]+ experiments["methods"] + experiments["features"]))
modelnames.sort()
filenames = glob.glob(f"{result_dir}{modelnames[0]}/*-Angry-0.0-0.wav")
filenames.sort()
for path in filenames[start:start+num]:
    basename = os.path.basename(path).split(".")[0]
    basename, _, _ = basename.split("-")
    if not (basename in evaluation_filenames):
        continue
    spk, name = basename.split("_")
    for emotion in emos:
        for mn in modelnames:
            if "msemotts" in mn:
                continue
            starter = "  {\n"
            starter += f'    "Name": "{basename}-{emotion}",\n'
            starter += f'    "TestID": "{mn}---{basename}---{emotion}",\n'
            starter += f'    "Text": "Intended Emotion: <b>{emotion}</b>",\n'
            starter += f'    "Files": {{\n'
            modelpaths = """"""
            for itt, intensity in enumerate(intensities):
                modelpath = result_dir + f"{mn}/{basename}-{emotion}-{intensity}-0.wav"
                added = f'      "{intensity_label[itt]}": "{modelpath}",\n'
                modelpaths += added
            modelpaths += f'    }}\n'
            modelpaths += f'  }},\n'
            whole_text += starter+modelpaths
whole_text += "]\n}}"
print(whole_text[:-1])